## Import

In [1]:
import os, sys, inspect, time

import numpy as np
import torch 
import matplotlib.pyplot as plt
torch.multiprocessing.set_sharing_strategy('file_system')

import discrepancy, visualization
from algorithms import ABC_algorithms, TPABC, SMCABC, SMC2ABC, SNLABC, SNL2ABC
import distributions 
import scipy.stats as stats

import utils_os, utils_math

%load_ext autoreload
%autoreload 2

In [2]:
np.round(0.12345,3)

0.123

## Problem Definition

In [3]:
from problems.ABC_problems import ABC_Problem

class Neuronal_Problem(ABC_Problem):
    
    def __init__(self, data, N=200, n=50):
        
        assert N <= data['Y'].shape[0]
        assert data['Y'].ndim == 2
        assert data['X'].shape[0] == data['Y'].shape[0]
        
        self.N = N # number of posterior samples
#         subsample = np.random.choice(np.arange(data['Y'].shape[0]),size=N,replace=True)
        self.n = n # length of the data vector x = {x_1, ..., x_n}
        self.d = 1 # dimensionality of theta?
        self.prior_args = np.repeat(np.array([-np.inf,np.inf]).reshape((1,-1)),self.N,axis=0) # surprisingly, these are bounds on theta (on X in our casee)
        
        self.all_thetas = data['X']#[subsample]
        self.y_obs = data['Y']#[subsample]
        self.sim_accuracy = 4 # number of digits after a decimal point for theta
        self.sim = {np.round(data['X'][i],self.sim_accuracy): data['Y'][i] for i in range(data['X'].shape[0])} #here we use all!
        self.K = self.N # dimensionality of summary statistics???
        self.stat = 'raw' # raw means that sufficient statistics is unknown (I guess). y_obs = data_obs
        
        self.data_obs = self.y_obs #important that first dim=N & y_dim = product of these dims
    
    def get_true_theta(self):
        pass

    def sample_from_prior(self):
        return np.random.choice(self.all_thetas,size=self.N,replace=True) # just 1 sample, but may be a vector!!!!
    
    def simulator(self, theta):
        # instead of using the best nearest neighbour, use the best from a subsample FIX LATER
        y = np.empty((len(theta),self.y_obs.shape[1]))
        for i,t_raw in enumerate(theta):
            t = np.round(t_raw,self.sim_accuracy)
            if t in self.sim:
                y[i] = self.sim[t]
            else:
                subsample = np.random.choice(np.arange(self.all_thetas.shape[0]),size=self.N,replace=True)
                print('Subsampling for simulating new thetas!!!')
                discr = np.abs(self.all_thetas[subsample] - t)
                y[i] = self.sim[self.all_thetas[subsample][np.argmin(discr)]]
        return y # self.n x thetas (which is 1 o_O )

    # B. correlation between latent
    def _ss_corr(self, Z):
        V = np.mat(Z).T * np.mat(Z) / Z.shape[0]
        (d,d) = V.shape
        upper_tri_elements = V[np.triu_indices(d, k=1)]
        stat = np.array(upper_tri_elements)
        return stat
    
    def statistics(self, data, theta=None):
        if self.stat == 'raw':
            # (correlation) as summary statistics (NO MARGINALS in these data)
            stat = self._ss_corr(data)
            return stat
        else:
            raise NotImplementedError('No ground truth statistics')

In [4]:
import pickle as pkl
with open(f'/home/nina/CopulaGP/plos_fig5_data/ST260_Day1_Dataset.pkl',"rb") as f:
    data = pkl.load(f)
    
data['Y'] = data['Y'][:,:10]
print(data['Y'].shape)
    
problem = Neuronal_Problem(data)

DIR = 'results/Neuronal' 

(21471, 10)


In [5]:
# from nn import ISN, MSN, MAF, MDN

# def fit_vae(s):
#     print('\n > fitting encoder')
#     all_stats = torch.tensor(np.vstack(s.all_stats[0:s.l+1])).float().to(s.device)
#     all_samples = torch.tensor(np.vstack(s.all_samples[0:s.l+1])).float().to(s.device)
#     print(all_stats.shape,all_samples.shape,s.hidden_ratio)
#     [n, dim] = all_stats.size()
#     h = int(dim*s.hidden_ratio)
#     print('summary statistic dim =', h, 'original dim =', dim)
#     architecture = [dim] + [100, 100, h]    
#     print('architecture', architecture)
#     net = ISN.ISN(architecture, dim_y=s.problem.K, hyperparams=s.msn_hyperparams)
#     net.train().to(s.device)
#     net.learn(x=all_stats, y=all_samples)
#     net = net.eval().cpu()
#     s.vae_net = net
#     s.vae_array.append(net)

In [6]:
# class data_SNL2(SNL2ABC.SNL2_ABC):
#     def __init__(self, problem, discrepancy, hyperparams, **kwargs):
#         super(data_SNL2, self).__init__(problem, discrepancy, hyperparams, **kwargs)
        
#         self.needed_hyperparams = ['epsilon']
#         self.epsilon = hyperparams.epsilon
#         self.device = torch.device(hyperparams.device)
#         self.nde_net = None                             # the learned q(x|theta)
#         self.vae_net = None                             # the learned s(x)
#         self.nde_array = []                             
#         self.vae_array = []     
#         self.proposal_array = []                        # the proposal used at each round
#         self.hidden_ratio = hyperparams.hidden_ratio    # dimensionality of s.s 
#         self.msn_hyperparams = hyperparams
#         self.msn_hyperparams.y_obs = self.convert_stat(self.whiten(self.y_obs))
    
#     def simulate(self):

#         # > wrapper function of rejection sampling algorithm. Launch several processes to do sampling in parallel

#         # Initialization
#         self.stats = np.zeros((self.num_sim, self.y_dim), float)
#         self.samples = np.zeros((self.num_sim, self.num_theta), float)
#         self.discrepancies = []
        
#         for i in range(self.num_sim):
#             theta = self.prior()
#             y = self.problem.simulator(theta).reshape(-1)
#             self.samples[i] = theta
#             self.stats[i] = y
#             # Calculate error
#             error = self.discrepancy(self.y_obs, y)
#             self.discrepancies.append(error) # does it matter?

In [7]:
### Sequential Neural Likelihood + 
hyperparams = ABC_algorithms.Hyperparams()
hyperparams.save_dir = DIR
hyperparams.device = 'cuda:0'
hyperparams.num_sim = 1000                        # number of simulations # original can't be <1000 LOL.. bugs!
hyperparams.L = 10                                # number of learning rounds
hyperparams.hidden_ratio = 0.05                  # dimensionality of S(x)
hyperparams.type = 'plain'                       # the network architecture of S(x), use CNN here
hyperparams.estimator = 'JSD'                    # MI estimator; JSD or DC, see the paper
hyperparams.nde = 'MAF'                          # nde; MAF (D>1) or MDN (D=1)

snl2_abc = SNL2ABC.SNL2_ABC(problem, discrepancy=discrepancy.eculidean_dist, hyperparams=hyperparams)

# JSD_array = []
# for l in range(len(snl2_abc.nde_array)):
#     print('l=', l)
#     snl2_abc.set(l=l)
#     visualization.plot_likelihood(samples=true_samples, log_likelihood_function=snl2_abc.log_likelihood, dimensions=(0,1))
#     JSD = discrepancy.JSD(snl2_abc.log_likelihood, snl2_abc.log_likelihood, true_samples, true_samples, N_grid=30)
#     JSD_array.append(JSD)
#     print('JSD snl+ = ', JSD)

In [8]:
snl2_abc.run()


iteration  0
# of cpus =  4
[ABC] sub-process start![ABC] sub-process start!

[sampling] finished sampling  [ABC] sub-process start!
5[ABC] sub-process start!

[sampling] finished sampling  10
[sampling] finished sampling  15
[sampling] finished sampling  20

 > fitting encoder
summary statistic dim = 2 original dim = 45
architecture [45, 100, 100, 2]
validation size= 0.8
finished: t= 0 loss= 1.386307716369629 loss val= 1.3863115310668945 time= 0.07755160331726074
finished: t= 50 loss= 1.3862273693084717 loss val= 1.386296272277832 time= 0.039060354232788086
finished: t= 100 loss= 1.3859100341796875 loss val= 1.3862812519073486 time= 0.038402557373046875
finished: t= 150 loss= 1.3847376108169556 loss val= 1.3863153457641602 time= 0.03786921501159668
finished: t= 200 loss= 1.3820996284484863 loss val= 1.3861336708068848 time= 0.04541611671447754
finished: t= 250 loss= 1.3746299743652344 loss val= 1.3855609893798828 time= 0.041015625
finished: t= 300 loss= 1.3489980697631836 loss val= 1.

RuntimeError: cholesky_cpu: U(200,200) is zero, singular U.

In [ ]:
snl2_abs.convert_stat(data['Y']).shape

In [ ]:
all_stats = torch.tensor(np.vstack(snl2_abc.all_stats[0:snl2_abc.l+1])).float()
all_samples = torch.tensor(np.vstack(snl2_abc.all_samples[0:snl2_abc.l+1])).float()
print(all_samples.shape)
snl2_abc.vae_net.MI(all_stats,all_samples,n=10)

In [44]:
np.vstack(snl2_abc.all_samples[0:1]).shape

(100, 100)

In [36]:
all_samples = torch.tensor(data['Y']).float()
print(all_samples.shape)
snl2_abc.vae_net.encode2(all_samples)

torch.Size([21471, 10])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (21471x10 and 100x100)

## Inference

### SMC-ABC

In [13]:
## Sequential Monte Carlo ABC

hyperparams = ABC_algorithms.Hyperparams()
hyperparams.save_dir = DIR
hyperparams.device = 'cuda:0'
hyperparams.num_sim = 4000                        # number of simulations
hyperparams.num_samples = 150                     # number of samples to represent posterior
hyperparams.L = 2                                 # number of rounds in sequential learning

smc_abc = SMCABC.SMC_ABC(problem, discrepancy=discrepancy.eculidean_dist, hyperparams=hyperparams)
smc_abc.run()

JSD_smc_array = []
for l in range(hyperparams.L):
    print('round =', l)
    smc_abc.posterior = smc_abc.posterior_array[l]
    visualization.plot_likelihood(samples=true_samples, log_likelihood_function=smc_abc.log_likelihood, dimensions=(0,1))
    JSD = discrepancy.JSD(tp_abc.log_likelihood, smc_abc.log_likelihood, true_samples, true_samples, N_grid=30)
    JSD_smc_array.append(JSD)
    print('JSD smc = ', JSD)
utils_os.save_object(DIR, 'JSD_SMC', JSD_smc_array)

iteration  0
# of cpus =  4
[ABC] sub-process start!
[ABC] sub-process start!
[ABC] sub-process start!
[ABC] sub-process start!


Process ForkPoolWorker-21:
Process ForkPoolWorker-23:
Process ForkPoolWorker-22:

KeyboardInterrupt



In [ ]:
## Sequential Monte Carlo ABC +

hyperparams = ABC_algorithms.Hyperparams()
hyperparams.save_dir = DIR
hyperparams.device = 'cuda:1'
hyperparams.num_sim = 2000                       # number of simulations
hyperparams.num_samples = 150                    # number of samples to represent posterior
hyperparams.L = 10                                # number of learning rounds
hyperparams.hidden_ratio = 0.05                  # dimensionality of S(x)
hyperparams.type = 'cnn2d'                       # the network architecture of S(x), use CNN here
hyperparams.estimator = 'JSD'                    # MI estimator; JSD or DC, see the paper

smc2_abc = SMC2ABC.SMC2_ABC(problem, discrepancy=discrepancy.eculidean_dist, hyperparams=hyperparams)
smc2_abc.run()

JSD_smc2_array = []
for l in range(len(smc2_abc.posterior_array)):
    print('l=', l)
    smc2_abc.l = l
    smc2_abc.posterior = smc2_abc.posterior_array[l]
    visualization.plot_likelihood(samples=true_samples, log_likelihood_function=smc2_abc.log_likelihood, dimensions=(0,1))
    JSD = discrepancy.JSD(tp_abc.log_likelihood, smc2_abc.log_likelihood, true_samples, true_samples, N_grid=30)
    JSD_smc2_array.append(JSD)
    print('JSD smc2 = ', JSD)
utils_os.save_object(DIR, 'JSD_SMC2', JSD_smc2_array)

### SNL

In [ ]:
## Sequential Neural Likelihood
hyperparams = ABC_algorithms.Hyperparams()
hyperparams.save_dir = DIR
hyperparams.device = 'cuda:1'
hyperparams.num_sim = 1000
hyperparams.L = 10

print('\n SNL ABC')
snl_abc = SNLABC.SNL_ABC(problem, discrepancy=discrepancy.eculidean_dist, hyperparams=hyperparams)
snl_abc.run()

JSD_array = []
for l in range(len(snl_abc.nde_array)):
    print('l=', l)
    snl_abc.nde_net = snl_abc.nde_array[l]
    visualization.plot_likelihood(samples=true_samples, log_likelihood_function=snl_abc.log_likelihood, dimensions=(0,1))
    JSD = discrepancy.JSD(tp_abc.log_likelihood, snl_abc.log_likelihood, true_samples, true_samples, N_grid=30)
    JSD_array.append(JSD)
    print('JSD snl = ', JSD)
utils_os.save_object(DIR, 'JSD_SNL', JSD_array)